<a href="https://colab.research.google.com/github/timteo-cy/crypto-project/blob/main/crypto_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from datetime import date

In [ ]:
#Initialising API Key from Coin Gecko
from google.colab import userdata
cgapi = userdata.get('cgapi')

# Crypto Market Cycle Analysis
This project explores the dynamics of crypto market cycles using historical data on Bitcoin (BTC), Ethereum (ETH), and the top 10 altcoins (based on market cap) as of 21-09-2025.

Assets Covered:

BTC, ETH (benchmarks)

Altcoins: XRP, BNB, SOL, DOGE, ADA, TRX, LINK, HYPE, AVAX, SUI

**Initialisation of data from CoinGecko API into a single dataframe**

In [ ]:
# Get daily coin data for the past 365 days for all coins
# Ticker and API id was pulled out from the CoinGecko Token API List
coins = {
    'BTC': 'bitcoin',
    'ETH': 'ethereum',
    'XRP': 'ripple',
    'BNB': 'binancecoin',
    'SOL': 'solana',
    'DOGE': 'dogecoin',
    'ADA': 'cardano',
    'TRX': 'tron',
    'LINK': 'chainlink',
    'HYPE': 'hyperliquid',
    'AVAX': 'avalanche-2',
    'SUI': 'sui'
}

df_coins = []

for ticker, api_id in coins.items():
    url = f"https://api.coingecko.com/api/v3/coins/{api_id}/market_chart"
    headers = {"x-cg-demo-api-key": cgapi }
    response = requests.get(url, headers=headers, params = {"vs_currency":"usd", "days": 365})
    data = response.json()
    df_price = pd.DataFrame(data['prices'] , columns=['unix_timestamp', 'price'])
    df_mcap = pd.DataFrame(data['market_caps'] , columns=['unix_timestamp', 'market_cap'])
    df_vol = pd.DataFrame(data['total_volumes'] , columns=['unix_timestamp', 'vol_24hr'])

    df = df_price.merge(df_mcap, on='unix_timestamp').merge(df_vol, on='unix_timestamp')

    df['date'] = pd.to_datetime(df['unix_timestamp'], unit='ms').dt.date
    df['ticker'] = ticker
    if ticker == 'BTC':
      df['type'] = 'BTC'
    elif ticker == 'ETH':
      df['type'] = 'ETH'
    else:
      df['type'] = 'Altcoin'
    df.drop(columns='unix_timestamp', inplace=True)
    df_coins.append(df)

df = pd.concat(df_coins).reset_index()
df.drop(columns = 'index', inplace = True)
df.sort_values(['ticker','date']).reset_index(drop = True)
df = df[df['date'] < date.today() ]


**Calculating Daily and 7-Day Rolling Returns**

In [ ]:
df_returns = df
df_returns['daily_return'] = df_returns.groupby('ticker')['price'].pct_change() *100
df_returns['rolling_returns'] = df_returns.groupby('ticker')['daily_return'].rolling(window=7).mean().reset_index(level=0, drop=True)
df_returns.dropna(inplace = True)

In [ ]:
df_returns.to_csv('crypto_returns.csv', index=False)

**Calculating Daily and 7-Day Rolling Returns, taking into consideration Altcoin 'weights' based on the market cap**

In [ ]:
df_weighted_returns = df_returns
if ('BTC', 'ETH') in df_weighted_returns['type']:
    df_weighted_returns['weight'] = 1
else:
    df_weighted_returns['weight'] = df_weighted_returns['market_cap'] / df.groupby(['date', 'type'])['market_cap'].transform('sum')


In [ ]:
df_weighted_returns['weighted_return'] = df_weighted_returns['daily_return'] * df['weight']
df_weighted_returns['weighted_rolling_return'] = df_weighted_returns['rolling_returns'] * df['weight']
df_weighted_returns = df_weighted_returns.groupby(['date', 'type'])[['weighted_return', 'weighted_rolling_return']].sum().reset_index()

In [ ]:
df_weighted_returns.to_csv('crypto_weighted_returns.csv', index = False)

**Calculating % Market Cap of total market for assets covered in analysis**

In [ ]:
df_mcap = df
df_mcap = df_mcap.groupby(['type', 'date'])['market_cap'].sum().reset_index()
df_mcap['total_daily_mcap'] = df_mcap.groupby('date')['market_cap'].transform('sum')
df_mcap['daily_%_mcap'] = (df_mcap['market_cap'] / df_mcap['total_daily_mcap'] * 100.0).round(2)


,type,date,market_cap,total_daily_mcap,daily_%_mcap
0,Altcoin,2024-09-30,2.681038e+11,1.886076e+12,14.21
1,Altcoin,2024-10-01,2.556639e+11,1.817819e+12,14.06
2,Altcoin,2024-10-02,2.457048e+11,1.743876e+12,14.09
3,Altcoin,2024-10-03,2.383836e+11,1.721657e+12,13.85
4,Altcoin,2024-10-04,2.355147e+11,1.718608e+12,13.70


In [ ]:
df_mcap.to_csv('crypto_coin_dominance.csv', index=False)

**Calculating Volume**